# Writing CUDA Kernels

## Objectives

By the time you complete this section you will be able to: 

* Write custom CUDA kernels in Python and launch them with an execution configuration.
* Utilize grid stride loops for working in parallel over large data sets and leveraging memory coalescing.
* Use atomic operations to avoid race conditions when working in parallel.


## The CUDA Programming Model

Ufuncs and generalized ufuncs are the easiest way in Numba to use the GPU, and present an parallel computing abstraction that requires minimal understanding of the CUDA programming model.  However, not all functions can be written as ufuncs.  Many problems require greater flexibility, in which case you want to write a *CUDA kernel*, the topic of this notebook. 

Fully explaining the CUDA programming model is beyond the scope of this notebook.  We highly recommend that everyone writing CUDA kernels with Numba take the time to read Chapters 1 and 2 of the CUDA C Programming Guide:

 * Introduction: http://docs.nvidia.com/cuda/cuda-c-programming-guide/index.html#introduction
 * Programming Model: http://docs.nvidia.com/cuda/cuda-c-programming-guide/index.html#programming-model

The programming model chapter gets a little in to C specifics, but familiarity with CUDA C can help write better CUDA kernels in Python.

For the purposes of this notebook, the most important thing is to understand this diagram:
![Thread Hierarchy](http://docs.nvidia.com/cuda/cuda-c-programming-guide/graphics/grid-of-thread-blocks.png "Thread Hierarchy (from CUDA C Programming Guide)")

We will be writing a *kernel* that decribes the execution of a single thread in this hierarchy.  The CUDA compiler and driver will execute our kernel across a *thread grid* that is divided into *blocks* of threads.  Threads within the same block can exchange data very easily during the execution of a kernel, whereas threads in different blocks should generally not communicate with each other (with a few exceptions).

Deciding the best size for the CUDA thread grid is a complex problem (and depends on both the algorithm and the specific GPU compute capability), but here are some very rough heuristics that we follow:

  * the size of a block should be a multiple of 32 threads, with typical block sizes between 128 and 512 threads per block.
  * the size of the grid should ensure the full GPU is utilized where possible.  Launching a grid where the number of blocks is 2x-4x the number of "multiprocessors" on the GPU is a good starting place.  Something in the range of 20 - 100 blocks is usually a good starting point.
  * The CUDA kernel launch overhead does depend on the number of blocks, so we find it best not to launch a grid where the number of threads equals the number of input elements when the input size is very big.  We'll show a pattern for dealing with large inputs below.

Each thread distinguishes itself from the other threads using its unique thread (`threadIdx`) and block (`blockIdx`) index values, which can be multidimensional if launched that way.

## A First Example

This all will sound a little overwhelming at first, so let's start with a concrete example.  Let's write our addition function for 1D NumPy arrays.  CUDA kernels are compiled using the `numba.cuda.jit` decorator (not to be confused with the `numba.jit` decorator for the CPU):

In [1]:
from numba import cuda

@cuda.jit
def add_kernel(x, y, out):
    tx = cuda.threadIdx.x # this is the unique thread ID within a 1D block
    ty = cuda.blockIdx.x  # Similarly, this is the unique block ID within the 1D grid

    block_size = cuda.blockDim.x  # number of threads per block
    grid_size = cuda.gridDim.x    # number of blocks in the grid
    
    start = tx + ty * block_size
    stride = block_size * grid_size # number of threads in each block * number of blocks in the grid

    # assuming x and y inputs are same length
    for i in range(start, x.shape[0], stride):
        out[i] = x[i] + y[i]

That's a lot more typing than our ufunc example, and it is much more limited: only works on 1D arrays, doesn't verify input sizes match, etc.  Most of the function is spent figuring out how to turn the block and grid indices and dimensions into unique offsets into the input arrays.  The pattern of computing a starting index and a stride is a common way to ensure that your grid size is independent of the input size.  The striding will maximize bandwidth by ensuring that threads with consecuitive indices are accessing consecutive memory locations as much as possible.  Thread indices beyond the length of the input (`x.shape[0]`, since `x` is a NumPy array) automatically skip over the for loop.

Also note that we did not need to specify a type signature for the CUDA kernel.  Unlike `@vectorize`, Numba can infer the type signature from the inputs automatically, and much more reliably.

Let's call the function now on some data:

In [2]:
import numpy as np

n = 5000000
x = np.arange(n).astype(np.float32)
y = 2 * x
out = np.empty_like(x)

threads_per_block = 128
blocks_per_grid = 30

add_kernel[blocks_per_grid, threads_per_block](x, y, out)
print(out[:10])

/home/noeliarico/anaconda3/envs/testing_cuda/lib/python3.10/site-packages/numba/cuda/compiler.py:726: NumbaPerformanceWarning: Grid size (30) < 2 * SM count (164) will likely result in GPU under utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))


[ 0.  3.  6.  9. 12. 15. 18. 21. 24. 27.]


/home/noeliarico/anaconda3/envs/testing_cuda/lib/python3.10/site-packages/numba/cuda/cudadrv/devicearray.py:885: NumbaPerformanceWarning: Host array used in CUDA kernel will incur copy overhead to/from device.
  warn(NumbaPerformanceWarning(msg))


The unusual syntax for calling the kernel function is designed to mimic the CUDA Runtime API in C, where the above call would look like:
```
add_kernel<<<blocks_per_grid, threads_per_block>>>(x, y, out)
```
The arguments within the square brackets define the size and shape of the thread grid, and the arguments with parentheses correspond to the kernel function arguments.

Note that, unlike the ufunc, the arguments are passed to the kernel as full NumPy arrays.  The kernel can access any element in the array it wants, regardless of its position in the thread grid.  This is why CUDA kernels are significantly more powerful that ufuncs.  (But with great power, comes a greater amount of typing...)

Numba includes [several helper functions](https://numba.readthedocs.io/en/latest/cuda/kernels.html#absolute-positions) to simplify the thread offset calculations above.  You can write the function much more simply as:

In [5]:
@cuda.jit
def add_kernel(x, y, out):
    start = cuda.grid(1)      # 1 = one dimensional thread grid, returns a single value
    stride = cuda.gridsize(1) # ditto. the number of blocks in the grid

    # assuming x and y inputs are same length
    for i in range(start, x.shape[0], stride):
        out[i] = x[i] + y[i]

As before, using NumPy arrays forces Numba to allocate GPU memory, copy the arguments to the GPU, run the kernel, then copy the argument arrays back to the host.  This not very efficient, so you will often want to allocate device arrays:

In [6]:
x_device = cuda.to_device(x)
y_device = cuda.to_device(y)
out_device = cuda.device_array_like(x)

In [7]:
%timeit add_kernel[blocks_per_grid, threads_per_block](x, y, out)

/home/noeliarico/anaconda3/envs/testing_cuda/lib/python3.10/site-packages/numba/cuda/compiler.py:726: NumbaPerformanceWarning: Grid size (30) < 2 * SM count (164) will likely result in GPU under utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))
/home/noeliarico/anaconda3/envs/testing_cuda/lib/python3.10/site-packages/numba/cuda/cudadrv/devicearray.py:885: NumbaPerformanceWarning: Host array used in CUDA kernel will incur copy overhead to/from device.
  warn(NumbaPerformanceWarning(msg))
/home/noeliarico/anaconda3/envs/testing_cuda/lib/python3.10/site-packages/numba/cuda/compiler.py:726: NumbaPerformanceWarning: Grid size (30) < 2 * SM count (164) will likely result in GPU under utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))


14.4 ms ± 115 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [8]:
%timeit add_kernel[blocks_per_grid, threads_per_block](x_device, y_device, out_device); out_device.copy_to_host()

2.47 ms ± 11.3 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


[NOELIA] If the data is already in the device the execution is only 2.47. So 12s in the previous piece of code where the numpy arrays in the host are used are dedicated to copying the data back and ford from the CPU to the GPU.

[NOELIA] In order to understand better the stride in the code, executing it with different data:

In [19]:
@cuda.jit
def first_kernel2(out):
    start = cuda.grid(1)      # 1 = one dimensional thread grid, returns a single value
    stride = cuda.gridsize(1) # ditto. the number of blocks in the grid

    # assuming x and y inputs are same length
    for i in range(start, x.shape[0], stride):
        out[i] = start


blocks_per_grid = 2
threads_per_block = 32
# stride = blocks_per_grid * threads_per_block
out_device = cuda.device_array_like(x)
first_kernel2[blocks_per_grid, threads_per_block](out_device)
# in out_device each position has the value of the thread_id that have updated it
print(out_device.copy_to_host())

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0.]
[ 0.  1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12. 13. 14. 15. 16. 17.
 18. 19. 20. 21. 22. 23. 24. 25. 26. 27. 28. 29. 30. 31. 32. 33. 34. 35.
 36. 37. 38. 39. 40. 41. 42. 43. 44. 45. 46. 47. 48. 49. 50. 51. 52. 53.
 54. 55. 56. 57. 58. 59. 60. 61. 62. 63.  0.  1.  2.  3.  4.  5.  6.  7.
  8.  9. 10. 11. 12. 13. 14. 15. 16. 17. 18. 19. 20. 21. 22. 23. 24. 25.
 26. 27. 28. 29. 30. 31. 32. 33. 34. 35.]


/home/noeliarico/anaconda3/envs/testing_cuda/lib/python3.10/site-packages/numba/cuda/compiler.py:726: NumbaPerformanceWarning: Grid size (2) < 2 * SM count (164) will likely result in GPU under utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))


## Kernel Synchronization

*One extremely important caveat should be mentioned here*: CUDA kernel execution is designed to be asynchronous with respect to the host program.  This means that the kernel launch `add_kernel[blocks_per_grid, threads_per_block](x_device, y_device, out_device)` returns immediately, allowing the CPU to continue executing while the GPU works in the background.  Only host<->device memory copies or an explicit synchronization call will force the CPU to wait until previously queued CUDA kernels are complete.

When you pass host NumPy arrays to a CUDA kernel, Numba has to synchronize on your behalf, but if you pass device arrays, processing will continue.  If you launch multiple kernels in sequence without any synchronization in between, they will be queued up to run sequentially by the driver, which is usually what you want.  If you want to run multiple kernels on the GPU in parallel (sometimes a good idea, but beware of race conditions!), take a look at [CUDA streams](https://numba.readthedocs.io/en/latest/cuda-reference/host.html#stream-management).

Here's some sample timings (using `%time`, which only runs the statement once to ensure our measurement isn't affected by the finite depth of the CUDA kernel queue):

In [20]:
# CPU input/output arrays, implied synchronization for memory copies
%time add_kernel[blocks_per_grid, threads_per_block](x, y, out)

CPU times: user 76.2 ms, sys: 8.6 ms, total: 84.8 ms
Wall time: 83.5 ms


/home/noeliarico/anaconda3/envs/testing_cuda/lib/python3.10/site-packages/numba/cuda/compiler.py:726: NumbaPerformanceWarning: Grid size (2) < 2 * SM count (164) will likely result in GPU under utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))
/home/noeliarico/anaconda3/envs/testing_cuda/lib/python3.10/site-packages/numba/cuda/cudadrv/devicearray.py:885: NumbaPerformanceWarning: Host array used in CUDA kernel will incur copy overhead to/from device.
  warn(NumbaPerformanceWarning(msg))


In [21]:
# GPU input/output arrays, no synchronization (but force sync before and after)
cuda.synchronize()
%time add_kernel[blocks_per_grid, threads_per_block](x_device, y_device, out_device)
cuda.synchronize()

CPU times: user 476 µs, sys: 72 µs, total: 548 µs
Wall time: 309 µs


/home/noeliarico/anaconda3/envs/testing_cuda/lib/python3.10/site-packages/numba/cuda/compiler.py:726: NumbaPerformanceWarning: Grid size (2) < 2 * SM count (164) will likely result in GPU under utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))


In [22]:
# GPU input/output arrays, include explicit synchronization in timing
cuda.synchronize()
%time add_kernel[blocks_per_grid, threads_per_block](x_device, y_device, out_device); cuda.synchronize()

CPU times: user 612 µs, sys: 16.3 ms, total: 16.9 ms
Wall time: 16.9 ms


**Always be sure to synchronize with the GPU when benchmarking CUDA kernels!**

[NOELIA] In this three situations:
- 1 benchmarks also the time needed for copying the data to the device and back to the host
- 2 launches the kernel does not include the measurement of time required by the second cuda.synchronize: i.e. the measure of time finishes before the execution of the GPU because the CPU goes to the next line of the code
- 3 is the correct one because is stops in the second synchronization until the execution is over and this is included in the measurement

## Atomic Operations and Avoiding Race Conditions

CUDA, like many parallel execution frameworks, makes it possible to have race conditions in your code.  A race condition in CUDA arises when threads read or write a memory location that might be modified by another independent thread.  Generally speaking, you need to worry about:

 * read-after-write hazards: One thread is reading a memory location at the same time another thread might be writing to it.
 * write-after-write hazards: Two threads are writing to the same memory location, and only one write will be visible when the kernel is complete.
 
A common strategy to avoid both of these hazards is to organize your CUDA kernel algorithm such that each thread has exclusive responsibility for unique subsets of output array elements, and/or to never use the same array for both input and output in a single kernel call.  (Iterative algorithms can use a double-buffering strategy if needed, and switch input and output arrays on each iteration.)

However, there are many cases where different threads need to combine results.  Consider something very simple, like: "every thread increments a global counter."  Implementing this in your kernel requires each thread to:

1. Read the current value of a global counter.
2. Compute `counter + 1`.
3. Write that value back to global memory.

There is no guarantee that another thread has not changed the global counter between steps 1 and 3.  To resolve this problem, CUDA provides efficient "atomic operations" which will read, modify and update a memory location in one, indivisible step.  Numba supports several of these functions, [described here](http://numba.pydata.org/numba-doc/dev/cuda/intrinsics.html#supported-atomic-operations).

Let's make our thread counter kernel:

In [23]:
@cuda.jit
def thread_counter_race_condition(global_counter):
    global_counter[0] += 1  # This is bad
    
@cuda.jit
def thread_counter_safe(global_counter):
    cuda.atomic.add(global_counter, 0, 1)  # Safely add 1 to offset 0 in global_counter array

In [28]:
# This gets the wrong answer
global_counter = cuda.to_device(np.array([0], dtype=np.int32))
thread_counter_race_condition[64, 64](global_counter)

print('Should be %d:' % (64*64), global_counter.copy_to_host())

Should be 4096: [1]


In [29]:
# This works correctly
global_counter = cuda.to_device(np.array([0], dtype=np.int32))
thread_counter_safe[64, 64](global_counter)

print('Should be %d:' % (64*64), global_counter.copy_to_host())

Should be 4096: [4096]


## Exercise

For this exercise, create a histogramming kernel.  This will take an array of input data, a range and a number of bins, and count how many of the input data elements land in each bin.  Below is an example CPU implementation of histogramming:

In [15]:
def cpu_histogram(x, xmin, xmax, histogram_out):
    '''Increment bin counts in histogram_out, given histogram range [xmin, xmax).'''
    # Note that we don't have to pass in nbins explicitly, because the size of histogram_out determines it
    nbins = histogram_out.shape[0]
    bin_width = (xmax - xmin) / nbins
    
    # This is a very slow way to do this with NumPy, but looks similar to what you will do on the GPU
    for element in x:
        bin_number = np.int32((element - xmin)/bin_width)
        if bin_number >= 0 and bin_number < histogram_out.shape[0]:
            # only increment if in range
            histogram_out[bin_number] += 1

In [16]:
x = np.random.normal(size=10000, loc=0, scale=1).astype(np.float32)
xmin = np.float32(-4.0)
xmax = np.float32(4.0)
histogram_out = np.zeros(shape=10, dtype=np.int32)

cpu_histogram(x, xmin, xmax, histogram_out)

histogram_out

array([   3,   77,  480, 1565, 2900, 2844, 1576,  488,   58,    9],
      dtype=int32)

In [18]:
@cuda.jit
def cuda_histogram(x, xmin, xmax, histogram_out):
    '''Increment bin counts in histogram_out, given histogram range [xmin, xmax).'''
    # Replace this with your implementation
    # 1. Get your starting offset and stride
    start = cuda.grid(1)
    stride = cuda.gridsize(1) 
    nbins = histogram_out.shape[0]
    bin_width = (xmax - xmin) / nbins

    # 2. loop over data elements using starting offset and stride
    for i in range(start, x.shape[0], stride):
        # 3. Find the bin the input data falls into
        bin_number = np.int32((x[i] - xmin)/bin_width)
        # 4. If in range of the histogram (don't go off the ends of the array!), increment value
        if bin_number >= 0 and bin_number < histogram_out.shape[0]:
            cuda.atomic.add(histogram_out, bin_number, 1)  # add 1 in the bin_number position safely 

"""
histogram_out = np.zeros(shape=5, dtype=np.int32)
xmin = 1
xmax = 10
x = np.array([3,4,1,5,4,6,7,2,1,9]) 
# histogram_out: [3 3 2 1 1](1,1,2) (3,4,4) (5,6) (7) (9)
"""

histogram_out = np.zeros(shape=10, dtype=np.int32)
cuda_histogram[64, 64](x, xmin, xmax, histogram_out) 
print(histogram_out)

[   3   77  480 1565 2900 2844 1576  488   58    9]


/home/noeliarico/anaconda3/envs/testing_cuda/lib/python3.10/site-packages/numba/cuda/compiler.py:726: NumbaPerformanceWarning: Grid size (64) < 2 * SM count (164) will likely result in GPU under utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))
/home/noeliarico/anaconda3/envs/testing_cuda/lib/python3.10/site-packages/numba/cuda/cudadrv/devicearray.py:885: NumbaPerformanceWarning: Host array used in CUDA kernel will incur copy overhead to/from device.
  warn(NumbaPerformanceWarning(msg))
